In [3]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from menu_functions import menu_to_pdf

def rag_gpt():
    
    load_dotenv()
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
    
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    
    context = False
    question = ""
    
    while question != "Exit":
        question = input() # Save the question of the user
        if context == False:
            
            # Scraping du menu du restaurant
            menu_to_pdf(question)
            
            # Connection au contexte
            file_path = "menu.pdf"
            loader = PyPDFLoader(file_path)

            docs = loader.load()

            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
            splits = text_splitter.split_documents(docs)
            vectorstore = InMemoryVectorStore.from_documents(
                documents=splits, embedding=OpenAIEmbeddings()
            )

            retriever = vectorstore.as_retriever()

            # Prompt
            system_prompt = (
                "You are an assistant for question-answering tasks about the menu of a specific restaurant. "
                "A give you a menu, just answer my question, and don't take attention to the name that I specified. "
                "Use the following pieces of retrieved context to answer "
                "the question. If you don't know the answer, say that you "
                "don't know. Use three sentences maximum and keep the "
                "answer concise."
                "\n\n"
                "{context}"
            )

            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", system_prompt),
                    ("human", "{input}"),
                ]
            )

            question_answer_chain = create_stuff_documents_chain(llm, prompt)
            rag_chain = create_retrieval_chain(retriever, question_answer_chain)
            
            context = True

        if question != "Exit":
            # Write an answer from the LLM
            results = rag_chain.invoke({"input": question})
    
            # Print the answer
            print(results["answer"])

C:\Users\busch\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
rag_gpt()

 What is the menu at Esplanade ?


The menu at Esplanade includes student menus on the 1st and 2nd floors with dishes like Blue cord with homemade brown sauce, Carrot duo, Fries, Curry pasta gratin, homemade vegetables, Home-made lemon-flavoured chicken casserole, Rice pilaf, Torsades with homemade carbonara sauce, and Curry of chickpeas with homemade vegetables. Desserts offered on both floors are Paris-Brest, Lemon tart, and Apple compote with speculos. In the Crystal Shop (DRC) Cafeteria, dishes like Home blue cord, Carrot duo, Curry of chickpeas with homemade vegetables, and Rice pilaf are available for on-site consumption or take-out.


 What kind of dessert can I find here ?


At the Esplanade restaurant, you can find desserts like Paris-Brest, lemon tart, and apple compote with speculos.


 Is there some vegetebale today ?


Yes, there are vegetable options available today, such as carrot duo, homemade vegetables, and a vegetable soup of the day with carrots and celery.


 Exit
